In [32]:
import pandas as pd
import numpy as np

def calculate(category, tipe):
    df = pd.read_csv('Results/Sentiwordnet/hasil_'+ tipe + category +'.csv')
    tp, tn, fp, fn = 0, 0, 0, 0
    p = 'positive'
    n = 'negative'
    for index in range(0, len(df)):
        label = df['label'][index]
        predict = df['predict'][index]
        
        if label == p and predict == p:
            tp += 1
        elif label == n and predict == n:
            tn += 1
        elif label == n and predict == p:
            fp += 1
        elif label == p and predict == n:
            fn += 1
    
    precision = round(tp / (tp + fp), 2)
    recall = round(tp / (tp + fn), 2)
    f1 = round(2 * ((precision * recall) / (precision + recall)), 2)
    accuracy = round((tp+tn)/(tp+tn+fp+fn), 2)
    print(tp, '\t', tn, '\t', fp, '\t', fn, '\t', '\t', precision, '\t', recall, '\t', f1, '\t', '\t', accuracy)
    return tp, tn, fp, fn, precision, recall, f1, accuracy

In [33]:
def preprocessing(sentence):
    return sentence.lower()

def calculate_f1(p, r):
    return round(2*((p*r)/(p+r)),5)

negation = [
    "afraid",
    "can't",
    "cannot",
    "deny",
    "mean",
    "negate",
    "negation",
    "negative",
    "neither",
    "never",
    "no",
    "non",
    "none",
    "nor",
    "not",
    "nothing",
    "refusal",
    "refuse",
    "reject",
    "rejection",
#     "don't",
#     "shouldn't",
#     "wouldn't",
#     "didn't",
#     "though",
#     "although",
#     "wasn't",
#     "isn't",
#     "but"
]

In [34]:
import numpy as np

def print_hasil(tipe):
    print('tp', '\t', 'tn', '\t', 'fp', '\t', 'fn', '\t', '\t', 'prec', '\t', 'rec', '\t', 'f1', '\t', '\t', 'accuracy')
    precision = []
    recall = []
    f1 = []
    cat = ['AMBIENCE', 'FOOD', 'SERVICE', 'PRICES']
    for c in cat:
        a, b, c, d, e, f, g, h = calculate(c, tipe)
        precision.append(e)
        recall.append(f)
        f1.append(g)
#         sf = sf.append({'TP':a, 'TN':b, 'FP':c, 'FN':d, 'precision':e, 'recall':f, 'f1':g, 'accuracy': h}, ignore_index=True)
#         sf.to_excel("Results/Calculation/"+ tipe +"-swn.xlsx")
    print('', '\t', '', '\t', '', '\t', '', '\t', '\t', round(np.mean(precision),2), '\t', round(np.mean(recall),2), '\t', calculate_f1(round(np.mean(precision),2), round(np.mean(recall),2)), '\t', '\t')

In [35]:
import string

def predict_lexicon(opinions, positive_lexicon, negative_lexicon, sent):
    endscore = 0
    for opinion in opinions:
        if opinion in positive_lexicon:
            endscore += 1
        elif opinion in negative_lexicon:
            endscore -= 1
                
        
        words = sent.split(' ')
        word_around = []
        for x in range(0, len(words)):
            if words[x] in string.punctuation:
                continue
            try:
                if (words[x+1] == opinion) or (words[x+2] == opinion) or (words[x+3] == opinion) or (words[x+4] == opinion):
                    word_around.append(words[x])
                elif (words[x-1] == opinion):
                    word_around.append(words[x])
            except:
                pass
        for neg in negation:
            if neg in word_around:
                endscore *= (-1)
                break

    if endscore > 0:
        polarity = 'positive'
    else:
        polarity = 'negative'
           
    return polarity

In [36]:
import pandas as pd
def run(category):
    df = pd.read_csv('Results/'+ category +'.csv')
    sf = pd.DataFrame(columns = ['id','review', 'opinion', 'label', 'predict'])
    
    of = pd.read_csv('Results/Opinion/' + category + '.csv')
    positive_lexicon = []
    negative_lexicon = []
    
    for i in range(0, len(of)):
        if of['sentimen'][i] == 'positive':
            positive_lexicon.append(of['opini'][i])
        else:
            negative_lexicon.append(of['opini'][i])
            
    for index in range(0, len(df)):
        opinion = []
        if type(df['opinion'][index]) != float:
            opinion = df['opinion'][index].split('|')
            
        
            
        prediction = predict_lexicon(opinion, positive_lexicon, negative_lexicon, preprocessing(df['review'][index]))
        sf = sf.append({'id': df['id'][index], 
            'review': df['review'][index],
            'opinion': df['opinion'][index],
            'label': df['label'][index],
            'predict': prediction
           }, ignore_index=True)
    
    sf.to_csv("Results/Sentiwordnet/hasil_cir_"+ category +".csv")
    sf.to_excel("Results/Sentiwordnet/hasil_cir_"+ category +".xlsx")

In [37]:
run('AMBIENCE')
run('FOOD')
run('SERVICE')
run('PRICES')

In [38]:
print_hasil('cir_')

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
75 	 10 	 8 	 18 	 	 0.9 	 0.81 	 0.85 	 	 0.77
116 	 19 	 17 	 25 	 	 0.87 	 0.82 	 0.84 	 	 0.76
72 	 22 	 16 	 10 	 	 0.82 	 0.88 	 0.85 	 	 0.78
37 	 29 	 9 	 7 	 	 0.8 	 0.84 	 0.82 	 	 0.8
 	  	  	  	 	 0.85 	 0.84 	 0.84497 	 	
